# Basic Usage on UDA Benchmark Suite

## Load and view the Q&A labels

The Q&A labels are accessible through the csv files in the `dataset/qa` directory or by loading the dataset from the HuggingFace repository `qinchuanhui/UDA-QA`.

In [2]:
# Get the q&a labels from the csv file
import pandas as pd
from uda.utils import preprocess

DATASET_NAME = "tat"

csv_file_path = f"./dataset/qa/{DATASET_NAME}_qa.csv"
df = pd.read_csv(csv_file_path, sep="|")
qas_dict = preprocess.qa_df_to_dict(DATASET_NAME, df)


In [ ]:
# Or you can get the q&a labels from the huggingface dataset
import pandas as pd
from uda.utils import preprocess
from datasets import load_dataset

DATASET_NAME = "tat"

hf_dataset = load_dataset("qinchuanhui/UDA-QA", DATASET_NAME)
hf_data = hf_dataset["test"]
df = hf_data.to_pandas()
qas_dict = preprocess.qa_df_to_dict(DATASET_NAME, df)

In [3]:
# View the snapshot of qas_dict
for key in list(qas_dict.keys())[:2]:
    print("Document Name: ", key)
    print("Its Q&A pairs: ", qas_dict[key][:2])
    print("=========================================")

Document Name:  overseas-shipholding-group-inc_2019
Its Q&A pairs:  [{'question': 'What benefits are provided by the company to qualifying domestic retirees and their eligible dependents?', 'answers': {'answer': ['certain postretirement health care and life insurance benefits'], 'answer_type': 'span', 'answer_scale': nan}, 'q_uid': 'bbdcf6da614f34fdb63995661c81613f'}, {'question': 'What is the change in Interest cost on benefit obligation for pension benefits from December 31, 2018 and 2019?', 'answers': {'answer': ['129'], 'answer_type': 'arithmetic', 'answer_scale': nan}, 'q_uid': '0bf2a781ac6044d4d9dd94bd6cc1f790'}]
Document Name:  lifeway-foods-inc_2019
Its Q&A pairs:  [{'question': 'What was the low sale price per share for each quarters in 2018 in chronological order?', 'answers': {'answer': ['$ 5.99', '$ 4.79', '$ 2.66', '$ 1.88'], 'answer_type': 'multi-span', 'answer_scale': nan}, 'q_uid': 'f4c8e2d0155ac338249d0fe6feba49ac'}, {'question': "What is the symbol of the company's co

## Prepare the document data

In [4]:
# Get the local path of the example pdf file
example_doc_name = list(qas_dict.keys())[0]
example_qas = qas_dict[example_doc_name][:3]
pdf_path = preprocess.get_example_pdf_path(
    DATASET_NAME, example_doc_name
)  # the function can be exchangable with get_pdf_path()
if pdf_path is None:
    print("No pdf found for this document")

### Basic data extraction:**

Leverage the library `PyPDF` to extract the raw text data from the pdf_files. The tabular structure are presented as the structural markers, such as line-breakers and space.

In [8]:
import PyPDF2
# Extract text from pdf
pdf_text = ""
with open(pdf_path, "rb") as file:
    # Create a PDF file reader object
    reader = PyPDF2.PdfReader(file, strict=False)
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        pdf_text += page.extract_text()
# Show a snapshot of the text
print(pdf_text[8000:8500])


hington, D.C. 20549 (information on the operation of the Public Reference Room is available by calling the SEC
at 1-800-SEC-0330). The SEC also maintains a website that contains reports, proxy and information statements, and other
information regarding issuers that file electronically with the SEC at http://www.sec.gov.
 
The Company also makes available on its website its corporate governance guidelines, its code of business conduct, insider trading
policy, anti-bribery and corruption policy an


### Data segmentation

Utilize the `langchain.text_splitter` to recursively segment text into overlapping chunks, maintaining a 10% overlap and taking explicit separators such as `\n\n` and `\n` into account.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,  # chunk size in characters not in words
    chunk_overlap=300,  # no overlap
)
text_chunks = text_splitter.split_text(pdf_text)
print("chunk_num:", len(text_chunks))
avg_chunk_word_counts = sum([len(chunk.split()) for chunk in text_chunks]) / len(text_chunks)
print("chunk_word_counts:", avg_chunk_word_counts)


chunk_num: 143
chunk_word_counts: 444.2237762237762


## Conduct indexing and retrieval
In this demo, we utilize the traditional dense embedding approach, utilizing the prevalent `SentenceTransformer` framework, specifically the `all-MiniLM-L6` model, within the vector database `ChromaDB`. 

Both queries and document segments are embedded into vectors, upon which cosine similarity measures are computed to retrieve the segments with the highest relevance.

In [ ]:
import chromadb
import torch
import chromadb.utils.embedding_functions as embedding_functions

# Create the vector_db collection 
# and store the embeddings
model_name = "all-MiniLM-L6-v2"
chroma_client = chromadb.Client()
device_info = "cuda" if torch.cuda.is_available() else "cpu"
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=model_name, device=device_info
)
collection = chroma_client.create_collection(
    "demo_vdb", embedding_function=ef, metadata={"hnsw:space": "cosine"}
)
id_list = [str(i) for i in range(len(text_chunks))]
collection.add(documents=text_chunks, ids=id_list)

In [19]:
# Fetch the top_k most similar chunks according to the query
top_k = 5
question = example_qas[0]["question"]
fetct_res = collection.query(query_texts=[question], n_results=top_k)
contexts = fetct_res["documents"][0]

# Show a snapshot of the context
print(f"The most relevant contexts to the question: {question}")
for idx,context in enumerate(contexts):
    print(f"===== Context {idx+1} =======")
    print(context[:200], "...")


The most relevant contexts to the question: What benefits are provided by the company to qualifying domestic retirees and their eligible dependents?
===== Context 1 =======
the five consecutive plan years that produce the highest results.
 
Multiemployer Pension and Postretirement Benefit Plans
 
The Company’s subsidiaries are parties to collective-bargaining agreements  ...
===== Context 2 =======
employee contributions and matching contributions to the plans. All contributions to the plans are at the discretion of the Company.
The Company’s contributions to the plan were $2,414 and $1,956 for  ...
===== Context 3 =======
underfunded multiemployer pension plan would require us to make payments to the plan for our proportionate share of such
multiemployer pension plan’s unfunded vested liabilities. See Note 16, “Pension ...
===== Context 4 =======
estimates and key assumptions, including those related to the discount rates, the rates expected to be earned on investments of plan
assets 

## Perform LLM answering
We input the combination of contexts and the question into a LLM to generate the final response. This process was illustrated utilizing locally-hosted open-source LLMs as well as commercially available GPT models.

In [26]:
# Example on GPT model
from uda.utils import llm
from openai import AzureOpenAI

llm_type = "gpt-4"
# Create the prompt tailored for different datasets and LLMs  
context_text = "\n".join(contexts)
llm_message = llm.make_prompt(question=question, context=context_text, task_name=DATASET_NAME, llm_type=llm_type)

# Call GPT-4/GPT-3.5 through Azure OpenAI API
# You should replace the following variables with your own configurations
# You can also use other API platforms here
your_api_key = "abcdefg"
your_endpoint = "https://abcdefg.openai.azure.com/"
your_deploy_model = "gpt-4"

client = AzureOpenAI(
    api_key = your_api_key,
    api_version = "2024-04-01-preview",
    azure_endpoint = your_endpoint,
)
raw_response = client.chat.completions.create(
    model = your_deploy_model, 
    messages = llm_message,
    temperature = 0.1,
)
response = raw_response.choices[0].message.content

# Show the response
print(f"Question: {question}")
print(f"Ground Truth Reference: {example_qas[0]['answers']}")
print(f"LLM Response: {response}")

Question: What benefits are provided by the company to qualifying domestic retirees and their eligible dependents?
Ground Truth Reference: {'answer': ['certain postretirement health care and life insurance benefits'], 'answer_type': 'span', 'answer_scale': nan}
LLM Response: The answer is: Postretirement health care and life insurance benefits.


In [ ]:
# Example on Llama model
from uda.utils import llm
from uda.utils import inference

llm_name = "meta-llama/Meta-Llama-3-8B-Instruct"
llm_type = "llama-8B"
# Create the prompt tailored for different datasets and LLMs  
context_text = "\n".join(contexts)
llm_message = llm.make_prompt(question=question, context=context_text, task_name=DATASET_NAME, llm_type=llm_type)

# Local Inference
llm_service = inference.LLM(llm_name)
llm_service.init_llm()
response = llm_service.infer(llm_message)

# Show the response
print(f"Question: {question}")
print(f"Ground Truth Reference: {example_qas[0]['answers']}")
print(f"LLM Response: {response}")
